In [1]:
%reload_ext autoreload
%autoreload 1

In [2]:
import numpy as np

In [3]:
%aimport malariagen_data.ag3

In [4]:
%aimport malariagen_data.veff 

In [5]:
veff = malariagen_data.veff

In [6]:
ag3 = malariagen_data.Ag3("gs://vo_agam_release")

In [7]:
genome = ag3._open_genome()
genome

<zarr.hierarchy.Group '/'>

In [8]:
list(genome)

['2L', '2R', '3L', '3R', 'Mt', 'UNKN', 'X', 'Y_unplaced']

In [9]:
genome["3R"][0:10].tobytes().decode()

'CCTCTACGTT'

In [10]:
ann = veff.Annotator(
    genome=genome,
    gff3_path="gs://vo_agam_release/reference/genome/agamp4/Anopheles-gambiae-PEST_BASEFEATURES_AgamP4.12.gff3.gz",
)

In [11]:
ann.get_feature("AGAP004707-RD")

('2L',
 'VectorBase',
 'mRNA',
 2358158,
 2431617,
 '.',
 '+',
 '.',
 'AGAP004707-RD',
 'AGAP004707')

In [12]:
ann.get_children("AGAP004707")

[('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  '.',
  '+',
  '.',
  'AGAP004707-RA',
  'AGAP004707'),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  '.',
  '+',
  '.',
  'AGAP004707-RB',
  'AGAP004707'),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  '.',
  '+',
  '.',
  'AGAP004707-RC',
  'AGAP004707'),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  '.',
  '+',
  '.',
  'AGAP004707-RD',
  'AGAP004707'),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  '.',
  '+',
  '.',
  'AGAP004707-RE',
  'AGAP004707'),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  '.',
  '+',
  '.',
  'AGAP004707-RF',
  'AGAP004707'),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  '.',
  '+',
  '.',
  'AGAP004707-RG',
  'AGAP004707'),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  '.',
  '+',
  '.',
  'AGAP004707-RH',
  'AGAP004707'),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  '.',
  '+',
  '.',
  'AGAP004707-RI',


In [13]:
ann.find("2L", 2358158, 2358159)

[('2L', 'VectorBase', 'chromosome', 1, 49364325, '.', '.', '.', '2L', None),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  '.',
  '+',
  '0',
  'AGAP004707-PA',
  'AGAP004707-RA'),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  '.',
  '+',
  '0',
  'AGAP004707-PB',
  'AGAP004707-RB'),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  '.',
  '+',
  '0',
  'AGAP004707-PC',
  'AGAP004707-RC'),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  '.',
  '+',
  '0',
  'AGAP004707-PD',
  'AGAP004707-RD'),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  '.',
  '+',
  '0',
  'AGAP004707-PE',
  'AGAP004707-RE'),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  '.',
  '+',
  '0',
  'AGAP004707-PF',
  'AGAP004707-RF'),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  '.',
  '+',
  '0',
  'AGAP004707-PG',
  'AGAP004707-RG'),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  '.',
  '+',
  '0',
  'AGAP004707-PH',
  'AGAP004707-RH'),
 ('

In [14]:
ann.get_effects("2L", 2422652, "A", "T", transcript_ids=["AGAP004707-RA"])

<generator object get_effects at 0x7f68db5b65c8>

In [15]:
for effect in ann.get_effects("2L", 2422652, "A", "T", transcript_ids=["AGAP004707-RD"]):
    print(effect)

VariantEffect(effect='NON_SYNONYMOUS_CODING', impact='MODERATE', chrom='2L', pos=2422652, ref='A', alt='T', vlen=0, ref_start=2422652, ref_stop=2422652, gene_id='AGAP004707', gene_start=2358158, gene_stop=2431617, gene_strand='+', transcript_id='AGAP004707-RD', transcript_start=2358158, transcript_stop=2431617, transcript_strand='+', cds_id='AGAP004707-PD', cds_start=2422468, cds_stop=2422655, cds_strand='+', intron_start=None, intron_stop=None, intron_5prime_dist=None, intron_3prime_dist=None, intron_cds_5prime=None, intron_cds_3prime=None, ref_cds_start=2984, ref_cds_stop=2984, ref_intron_start=None, ref_intron_stop=None, ref_start_phase=2, ref_codon='ttA', alt_codon='ttT', codon_change='ttA/ttT', aa_pos=995, ref_aa='L', alt_aa='F', aa_change='L995F')


In [16]:
effect.effect

'NON_SYNONYMOUS_CODING'

In [17]:
df = ag3.snp_effects('AGAP004707-RA', 'gamb_colu')

transcript : AGAP004707-RA
chromosome : 2L 
start : 2358158
stop : 2431617
strand : +


In [18]:
df

,position,ref_allele,alt_alleles
0,2358158,A,C
1,2358158,A,T
2,2358158,A,G
3,2358159,T,A
4,2358159,T,C
...,...,...,...
132301,2431616,G,C
132302,2431616,G,T
132303,2431617,A,C
132304,2431617,A,T


In [19]:
for effect in ann.get_effects(chrom='2L', pos=2429745, ref='A', alt='T',
                                  transcript_ids=["AGAP004707-RA"]):
       print(effect)

VariantEffect(effect='NON_SYNONYMOUS_CODING', impact='MODERATE', chrom='2L', pos=2429745, ref='A', alt='T', vlen=0, ref_start=2429745, ref_stop=2429745, gene_id='AGAP004707', gene_start=2358158, gene_stop=2431617, gene_strand='+', transcript_id='AGAP004707-RA', transcript_start=2358158, transcript_stop=2431617, transcript_strand='+', cds_id='AGAP004707-PA', cds_start=2429556, cds_stop=2429801, cds_strand='+', intron_start=None, intron_stop=None, intron_5prime_dist=None, intron_3prime_dist=None, intron_cds_5prime=None, intron_cds_3prime=None, ref_cds_start=4545, ref_cds_stop=4545, ref_intron_start=None, ref_intron_stop=None, ref_start_phase=0, ref_codon='Aat', alt_codon='Tat', codon_change='Aat/Tat', aa_pos=1516, ref_aa='N', alt_aa='Y', aa_change='N1516Y')


In [24]:
df_short = df[:10]

In [25]:
test = []
for row in df_short.itertuples(index=True):
    for effect in ann.get_effects(chrom='2L', pos=row.position, ref=row.ref_allele, alt=row.alt_alleles,
                                  transcript_ids=["AGAP004707-RA"]):
        test.append(effect.effect)
        

In [26]:
test

['START_LOST',
 'START_LOST',
 'START_LOST',
 'NON_SYNONYMOUS_CODING',
 'NON_SYNONYMOUS_CODING',
 'NON_SYNONYMOUS_CODING',
 'NON_SYNONYMOUS_CODING',
 'NON_SYNONYMOUS_CODING',
 'NON_SYNONYMOUS_CODING',
 'NON_SYNONYMOUS_CODING']